In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split

In [ ]:
def load_data(data_dir):
    images = []
    labels = []
    
    # Iterate over the dataset folder and read images
    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)
        
        if not os.path.isdir(folder_path):
            continue
        
        label = 0 if folder_name == 'no_belt' else 1
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            img = cv2.imread(image_path)
            img = cv2.resize(img, (64, 64))  # Resize image to 64x64
            images.append(img)
            labels.append(label)
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

# Load the data (adjust path as needed)
data_dir = "path_to_seat_belt_data"
images, labels = load_data(data_dir)

# Normalize the images
images = images.astype("float32") / 255.0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Check the shape of the data
print(f"Training data shape: {X_train.shape}, Test data shape: {X_test.shape}")


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification: seat belt or not
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
model.save('seat_belt_detection_model.h5')


In [ ]:
# Real-time seat belt detection using OpenCV
def detect_seat_belt(frame, model):
    frame_resized = cv2.resize(frame, (64, 64))  # Resize to match input shape
    frame_resized = frame_resized.astype("float32") / 255.0  # Normalize
    frame_resized = np.expand_dims(frame_resized, axis=0)  # Add batch dimension
    
    prediction = model.predict(frame_resized)
    label = "Seat Belt On" if prediction[0] > 0.5 else "No Seat Belt"
    
    return label

# Start video capture
cap = cv2.VideoCapture(0)  # Use 0 for webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Get prediction for seat belt
    label = detect_seat_belt(frame, model)
    
    # Display result
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Seat Belt Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()